In [1]:
from dataclasses import dataclass
from typing import Dict, List, Optional


@dataclass
class EstadoEconomico:
    ano: int
    divida_nominal: float
    pib_nominal: float
    inflacao: float
    crescimento_real: float
    juros_nominal: float
    primario_frac_pib: float


class PoliticaFiscal:
    def __init__(self, pb_base: float, resposta_divida: float = 0.0, alvo_divida_pib: float = 0.0):
        self.pb_base = pb_base
        self.resposta_divida = resposta_divida
        self.alvo = alvo_divida_pib

    def primario_frac_pib(self, divida_pib: float) -> float:
        return self.pb_base + self.resposta_divida * (divida_pib - self.alvo)


class RegraJuros:
    def __init__(self, i_base: float, limiar: float, slope: float, teto: Optional[float] = None):
        self.i_base = i_base
        self.limiar = limiar
        self.slope = slope
        self.teto = teto

    def juros(self, divida_pib: float) -> float:
        premio = 0.0
        if divida_pib > self.limiar:
            premio = self.slope * (divida_pib - self.limiar)
        i = self.i_base + premio
        if self.teto is not None:
            i = min(i, self.teto)
        return i


@dataclass
class Choque:
    ano: int
    d_g: float = 0.0
    d_pi: float = 0.0
    d_pb: float = 0.0
    d_i: float = 0.0


class EconomiaDivida:
    def __init__(
        self,
        B0: float,
        Y0: float,
        g0: float,
        pi0: float,
        politica_fiscal: PoliticaFiscal,
        regra_juros: RegraJuros,
        choques: List[Choque] = None,
    ):
        self.B = B0
        self.Y = Y0
        self.g = g0
        self.pi = pi0
        self.fiscal = politica_fiscal
        self.juros_rule = regra_juros
        self.choques_por_ano: Dict[int, List[Choque]] = {}
        for c in (choques or []):
            self.choques_por_ano.setdefault(c.ano, []).append(c)

    def _aplicar_choques(self, ano: int, g: float, pi: float, pb: float, i: float):
        for c in self.choques_por_ano.get(ano, []):
            g += c.d_g
            pi += c.d_pi
            pb += c.d_pb
            i += c.d_i
        return g, pi, pb, i

    def passo(self, ano: int) -> EstadoEconomico:
        b = self.B / self.Y
        pb = self.fiscal.primario_frac_pib(b)
        i = self.juros_rule.juros(b)
        g, pi, pb, i = self._aplicar_choques(ano, self.g, self.pi, pb, i)
        Y_next = self.Y * (1.0 + g) * (1.0 + pi)
        PB_nominal = pb * self.Y
        B_next = (1.0 + i) * self.B - PB_nominal
        estado = EstadoEconomico(
            ano=ano,
            divida_nominal=self.B,
            pib_nominal=self.Y,
            inflacao=pi,
            crescimento_real=g,
            juros_nominal=i,
            primario_frac_pib=pb,
        )
        self.B, self.Y = B_next, Y_next
        self.g, self.pi = g, pi
        return estado


class Simulador:
    def __init__(self, economia: EconomiaDivida):
        self.economia = economia

    def rodar(self, ano_inicial: int, anos: int) -> List[Dict]:
        out: List[Dict] = []
        for k in range(anos):
            ano = ano_inicial + k
            e = self.economia.passo(ano)
            b = e.divida_nominal / e.pib_nominal
            out.append({
                "ano": e.ano,
                "divida_pib": b,
                "juros_nominal": e.juros_nominal,
                "crescimento_real": e.crescimento_real,
                "inflacao": e.inflacao,
                "primario_frac_pib": e.primario_frac_pib,
                "B": e.divida_nominal,
                "Y": e.pib_nominal,
            })
        return out


if __name__ == "__main__":
    Y0 = 100.0
    B0 = 80.0

    g0 = 0.02
    pi0 = 0.04

    fiscal = PoliticaFiscal(pb_base=0.01, resposta_divida=0.05, alvo_divida_pib=0.70)
    juros = RegraJuros(i_base=0.08, limiar=0.85, slope=0.20, teto=0.25)

    choques = [
        Choque(ano=2027, d_g=-0.03),
        Choque(ano=2027, d_pb=-0.01),
        Choque(ano=2028, d_pi=0.03),
    ]

    economia = EconomiaDivida(
        B0=B0,
        Y0=Y0,
        g0=g0,
        pi0=pi0,
        politica_fiscal=fiscal,
        regra_juros=juros,
        choques=choques
    )

    sim = Simulador(economia)
    res = sim.rodar(ano_inicial=2026, anos=10)

    for r in res:
        print(
            f"{r['ano']} | b={r['divida_pib']:.3f} | "
            f"i={100*r['juros_nominal']:.1f}% | "
            f"g={100*r['crescimento_real']:.1f}% | "
            f"pi={100*r['inflacao']:.1f}% | "
            f"pb={100*r['primario_frac_pib']:.2f}%"
        )
r

2026 | b=0.800 | i=8.0% | g=2.0% | pi=4.0% | pb=1.50%
2027 | b=0.800 | i=8.0% | g=-1.0% | pi=4.0% | pb=0.50%
2028 | b=0.835 | i=8.0% | g=-1.0% | pi=7.0% | pb=1.67%
2029 | b=0.835 | i=8.0% | g=-1.0% | pi=7.0% | pb=1.68%
2030 | b=0.836 | i=8.0% | g=-1.0% | pi=7.0% | pb=1.68%
2031 | b=0.836 | i=8.0% | g=-1.0% | pi=7.0% | pb=1.68%
2032 | b=0.837 | i=8.0% | g=-1.0% | pi=7.0% | pb=1.68%
2033 | b=0.837 | i=8.0% | g=-1.0% | pi=7.0% | pb=1.69%
2034 | b=0.838 | i=8.0% | g=-1.0% | pi=7.0% | pb=1.69%
2035 | b=0.838 | i=8.0% | g=-1.0% | pi=7.0% | pb=1.69%
